## Models

- BiLSTM
- BiLSTM CRF


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-5d3_qube
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-5d3_qube
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=d05e339766de72d176775615478da9684e3afd00b064dcaaa70275dd5c97baa8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ob1gey18/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [ ]:
# downgrade to tensorflow version 2.2 and keras version 2.3.1
# !pip list -> shows that keras has version 2.15.0 and tensorflow has 2.15.0
!pip uninstall tensorflow -y
!pip uninstall keras -y
!pip install tensorflow==2.2  # Replace x.y.z with your desired version
!pip install keras==2.3.1       # Replace x.y.z with your desired version

Found existing installation: Keras 2.3.1
Uninstalling Keras-2.3.1:
  Successfully uninstalled Keras-2.3.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 64.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-a

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Bidirectional, LSTM, SimpleRNN
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, Lambda
from keras_contrib.layers import CRF
import keras.optimizers as ko
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import keras
import subprocess
from sklearn.utils import class_weight

Using TensorFlow backend.


ModuleNotFoundError: No module named 'keras.api'

In [ ]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = 'drive/My Drive/Experiment Design/'
file_path = PATH + "data/Task1.csv"
data = pd.read_csv(file_path)
data = data.rename(columns={'id':'Sentence #'})
data = data.drop('Unnamed: 0',axis=1)
data = data.fillna(method="ffill")

In [ ]:
data.groupby('labels').count()

,Sentence #,words
labels,,
B_EXC,1176,1176
B_INC,1223,1223
EXC,5713,5713
INC,5455,5455
O,29976,29976


In [ ]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["words"].values.tolist(),
                                                           s["labels"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None


words = list(set(data["words"].values))
tags = ['O','B_INC','INC','B_EXC','EXC']
# tags = list(set(data["labels"].values))
n_words = len(words)
n_tags = len(tags)

getter = SentenceGetter(data)
sentences = getter.sentences

word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
max_len = 170
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y = [np.array([tag2idx[w[1]] for w in s]) for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
len(sentences)

2154

In [ ]:
def loadGloveModel(File):
    print("Loading Glove Model")
    gloveModel = {}
    with open(File, 'r', encoding='utf-8') as f:
        for line in f:
            splitLines = line.split()
            word = splitLines[0]
            wordEmbedding = np.array([float(value) for value in splitLines[1:]])
            gloveModel[word] = wordEmbedding
    print(len(gloveModel), "words loaded!")
    return gloveModel

file_path = PATH + "data/glove.6B.200d.txt"
vec_model = loadGloveModel(file_path)


Loading Glove Model
400000 words loaded!


In [ ]:
emb_dim = len(vec_model['the'])
embedding_matrix = np.zeros((len(word2idx) + 1, emb_dim))

for word, i in word2idx.items():
    if word not in vec_model:
        continue
    embedding_vector = vec_model[word]
    embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(4916, 200)


In [ ]:
vec_model.clear()

In [ ]:
def write_results(result,file,ign):
    idx2tag = {i: w for w, i in tag2idx.items()}
    with open(file,'w+') as f:
        for i,lis in enumerate(result):
            line = ""
            for el in lis:
                tag = idx2tag[el]
                if tag in ['O',ign,'B_'+ign]:
                    line += "O "
                elif tag[0] == 'B':
                    line += 'B '
                else:
                    line += 'I '
            f.write(line+'\n')

def get_sampleWeights(class_weights):
    sample_weights=np.random.rand(X_train.shape[0], X_train.shape[1])
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[1]):
            sample_weights[i][j]=class_weights[np.argmax(y_train[i][j])]
    return sample_weights

def get_softMetrics(pred,labels,ign):

    write_results(pred,'pred.txt',ign)
    write_results(labels,'labels.txt',ign)
    out = subprocess.check_output(['./a.out']).decode('utf-8').split('\n')
    rows = []
    for item in  out:
        item = item.split(" ")
        if len(item) != 2:continue
        rows.append({'Proportional':item[0],'Binary':item[1]})

    return (pd.DataFrame(rows,index=['precision','recall','F1']))

def return_report(model,epochs):
    y_flat = list(np.argmax(y_test,2).flatten('F'))
#     class_weights = class_weight.compute_class_weight('balanced',y_flat)
    class_weights = [1,20,20,20,20]
    model.fit(X_train,y_train,epochs=epochs,verbose=1)
    out = model.predict(X_test)
    pred = np.argmax(out,2)
    labels = np.argmax(y_test,2)
    inc = get_softMetrics(pred,labels,'INC')
    exc = get_softMetrics(pred,labels,'EXC')
    display(inc)
    display(exc)
    report = classification_report(np.argmax(y_test,2).flatten('F'),pred.flatten('F'),output_dict=True)
    df = pd.DataFrame(report).transpose()
#     display(df)
    return pred
#     return report

In [ ]:
from keras.layers import Input, Embedding, LSTM, Bidirectional, TimeDistributed, Dense

def loss(y_true, y_pred):
    X = crf.input
    mask = crf.input_mask
    nloglik = crf.get_negative_log_likelihood(y_true, X, mask)
    return keras.activations.relu(nloglik)

def lstm_crf():
    input = Input(shape=(max_len,))
    model = Embedding(input_dim=n_words + 1, output_dim=emb_dim,
                      input_length=max_len, weights=[embedding_matrix],trainable=True)(input)  # 20-dim embedding
    model = Bidirectional(LSTM(units=emb_dim, return_sequences=True,
                               recurrent_dropout=0.1))(model)  # variational biLSTM

    model = TimeDistributed(Dense(2*emb_dim, activation="relu"))(model)  # a dense layer as suggested by neuralNer
    crf = CRF(n_tags,sparse_target=False)  # CRF layer
    out = crf(model)  # output

    adam = ko.Adam(lr=0.0008)
    sgd = ko.SGD(lr=0.05,momentum=0.7)
    rmsprop = ko.RMSprop(lr=0.001)
    model = Model(input, out)
    model.compile(optimizer='rmsprop', loss=crf.loss_function, metrics=[crf.accuracy])
    model.summary()
    return model

def get_bilstm_lstm_model():

    input = Input(shape=(max_len,))

    # Add Embedding layer
    model = Embedding(input_dim=n_words + 1, output_dim=emb_dim,
                  input_length=max_len, weights=[embedding_matrix],trainable=True)(input)

    # Add bidirectional LSTM
    model = Bidirectional(LSTM(units=emb_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.1))(model)

    # Add LSTM
    model = Bidirectional(LSTM(units=emb_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.1))(model)

    # Add timeDistributed Layer
    out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)

    #Optimiser
    adam = ko.Adam(lr=0.0009)

    # Compile model
    model = Model(input, out)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
from keras.optimizers import Adam, SGD, RMSprop
def lstm_crf():
    input = Input(shape=(max_len,))
    model = Embedding(input_dim=n_words + 1, output_dim=emb_dim,
                      input_length=max_len, weights=[embedding_matrix],trainable=True)(input)  # 20-dim embedding
    model = Bidirectional(LSTM(units=emb_dim, return_sequences=True,
                               recurrent_dropout=0.1))(model)  # variational biLSTM

    model = TimeDistributed(Dense(2*emb_dim, activation="relu"))(model)  # a dense layer as suggested by neuralNer
    crf = CRF(n_tags)  # CRF layer
    out = crf(model)  # output

    # update optimizer with learning_rate
    adam = Adam(learning_rate=0.0008)
    sgd = SGD(learning_rate=0.05, momentum=0.7)
    rmsprop = RMSprop(learning_rate=0.001)

    model = Model(input, out)
    model.compile(optimizer='rmsprop', loss=crf.loss_function, metrics=[crf.accuracy])
    model.summary()
    return model

In [ ]:
bilstm = get_bilstm_lstm_model()
labels = return_report(bilstm,1)

In [ ]:
crf = lstm_crf()
pred = return_report(crf,25)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 170)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 170, 200)          983200    
                                                                 
 bidirectional_1 (Bidirecti  (None, 170, 400)          641600    
 onal)                                                           
                                                                 
 time_distributed_1 (TimeDi  (None, 170, 400)          160400    
 stributed)                                                      
                                                                 
 crf_1 (CRF)                 (None, 170, 5)            2040      
                                                                 
Total params: 1787240 (6.82 MB)
Trainable params: 1787240 (

/usr/local/lib/python3.10/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.10/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


AttributeError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras_contrib/losses/crf_losses.py", line 54, in crf_loss  *
        crf, idx = y_pred._keras_history[:2]

    AttributeError: 'SymbolicTensor' object has no attribute '_keras_history'


In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b